# Tech Challenger 


In [1192]:
#!pip install numpy==1.26.4 pandas seaborn matplotlib scikit-learn statsmodels PrettyTable termcolor pmdarima arch yfinance xgboost prophet ipywidgets



In [ ]:
!pip  install ipywidgets


In [1194]:
import arch
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import pmdarima as pm
import scipy.stats as stats
from prophet import Prophet
from termcolor import colored
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [ ]:
df = pd.read_csv("Ibovespa 2004 a 2024.csv", sep=',')
df

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
serie_fechamento = pd.Series(data=df['Último'].values, index=pd.to_datetime(df['Data'], format='%d.%m.%Y'))
serie_fechamento = serie_fechamento.sort_index(ascending=True)
#serie_fechamento =  serie_fechamento.loc[pd.Timestamp('2014-01-02'):pd.Timestamp('2024-02-29')] #Obter os anos de 2014 a  fev 2024
serie_fechamento

Definindo tamanho padrão para plot do matplolib

In [1201]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 5

In [1202]:
def plotarSerie(serie, titulo="Valores da bolsa de valores com base no fechamento diário", legendas=[]):
    serie.plot()
    plt.title(titulo)
    plt.xlabel('Data')
    plt.ylabel('Valor em Real (R$)')
    if legendas:
        plt.legend(labels=legendas, loc='best')

In [ ]:
plotarSerie(serie_fechamento)

Média Móvel

In [ ]:
serie_media_movel_anual = serie_fechamento.rolling(365).mean()
serie_media_movel_anual.dropna().head()

In [1205]:
def plotar_medias_moveis(serie, titulo ='Valores da bolsa de valores com média movel' ):
    serie_media_movel_30 = serie.rolling(30-8).mean()
    serie_media_movel_90 = serie.rolling(90-(3*8)).mean()
    serie_media_movel_anual = serie.rolling(365-(12*8)).mean()
    plotarSerie(serie)
    plotarSerie(serie_media_movel_30)
    plotarSerie(serie_media_movel_90)
    plotarSerie(serie_media_movel_anual, titulo ,['Valor Real','Média movel 30 dias', 'Média Móvel 90 dias', 'Média Móvel Anual'])
    plt.show()

In [ ]:
plotar_medias_moveis(serie_fechamento)

In [ ]:
serie_fechamento

In [ ]:
stats.probplot(serie_fechamento, dist='norm', plot=plt)
plt.show()

In [ ]:
sns.histplot(serie_fechamento, kde=True)

**Teste Shapiro-Wilk**

### O que é o teste de Shapiro-Wilk?

    O teste de Shapiro-Wilk é uma ferramenta estatística utilizada para avaliar se um conjunto de dados se ajusta a uma distribuição normal. A distribuição normal, também conhecida como curva em forma de sino, é uma das distribuições mais comuns em estatística e é frequentemente utilizada em diversos modelos estatísticos.

#### Por que é importante testar a normalidade?

    Muitos testes estatísticos assumem que os dados seguem uma distribuição normal. Se essa premissa não for verdadeira, os resultados do teste podem ser enganosos. Ao realizar o teste de Shapiro-Wilk, você verifica se essa suposição é razoável para seus dados.

Critérios:

Nível de significância de 0,05 ou 5% (mais utilizado)

Valor de p < 0.05: Os dados provavelmente não são normalmente distribuídos.
Valor de p ≥ 0.05: Não há evidência suficiente para rejeitar a hipótese de normalidade.

In [1210]:
def validacao_distribuicao(serie):
    print(colored(':: Validação Distribuição (Shapiro-Wilk) ::', 'light_blue', attrs=["bold"]))
    e, p = stats.shapiro(serie)
    print(f'Estátistica do teste: {e}')
    print(f'p-valor: {p}')
    if p > e:
        print(colored('>> Distribuição Normal <<','green'))
    else:
        print(colored('Não há evidência suficiente para rejeitar a hipótese de normalidade','red'))


In [ ]:
validacao_distribuicao(serie_fechamento)

Iremos transformar os valores com o objetivo de diminuir a variancia  e melhorar a normalidade usando logaritmo

In [ ]:
serie_log = np.log(serie_fechamento)
serie_log.head()

In [ ]:
stats.probplot(serie_log, dist='norm', plot=plt)
plt.plot()

In [ ]:
sns.histplot(serie_log, kde=True)

In [ ]:
validacao_distribuicao(serie_log)

    - Comentário: Mesmo efetuando transformação em logaritmo não foi possível obter um bom resultado

Iremos elevar ao cubo para tentar obter uma melhor distribuição, usaremos o sign e abs para evitar o valores negativos

In [ ]:
serie_ao_cubo = np.sign(serie_fechamento)*abs(serie_fechamento)**(1/3)
serie_ao_cubo.head()

In [ ]:
stats.probplot(serie_ao_cubo, dist='norm', plot=plt)
plt.plot()

In [ ]:
sns.histplot(serie_ao_cubo, kde=True)

In [ ]:
validacao_distribuicao(serie_ao_cubo)

    - Comentário: Não foi possível obter a distribuição normal usando serie ao cubo

In [ ]:
serie_fechamento.isnull().value_counts()

In [ ]:
serie_decomposta = seasonal_decompose(serie_fechamento, period=30)
serie_decomposta.plot()
plt.tight_layout()

In [ ]:
serie_decomposta = seasonal_decompose(serie_fechamento, model="multiplicative", period=252)
serie_decomposta.plot()
plt.tight_layout()

In [ ]:
serie_decomposta.seasonal.iloc[-500:].plot()
plt.show()

Através da decomposição é possível notar que existe uma sazonalidade e uma tendencia de crescimento, que aparenta não ser estácionada.
Para validar iremos um o test de KPSS e Dict Fuller

Teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin)

Ho = não é estacionário: estatística do teste > valor crítico

Ha = é estacionário:  estatística do teste < valor crítico

In [1224]:
def validacao_kpss(serie):
    print(colored(':: Validação KPSS (Kwiatkowski-Phillips-Schmidt-Shin) ::', 'light_blue', attrs=["bold"]))
    resultado = kpss(serie)
    estatistica_teste = resultado[0]
    p_valor = resultado[1]
    valores_criticos  = resultado[3]
    percentil_referencia = '5%'

    print(f'Estatística do teste: {estatistica_teste:.4f}')
    print(f'p-valor {p_valor:.4f}')
    print('Valores Críticos:')
    table = PrettyTable(['Criticidade', 'Valor'])

    for chave, valor in valores_criticos.items():
        table.add_row([chave, valor])
    
    print(table)

    if estatistica_teste > valores_criticos[percentil_referencia]:
        print(colored('>> Serie não estacionária <<', 'red'))
    else:
        print(colored('>> Série estácionaria <<', 'green'))


In [ ]:
validacao_kpss(serie_fechamento)

In [ ]:
plot_acf(serie_fechamento)
plot_pacf(serie_fechamento)
plt.show()

# Aplicando Diferenciação

- Descrever como funciona a difenciação

In [ ]:
serie_fechamento_diff = serie_fechamento.diff()

serie_fechamento_diff = serie_fechamento_diff.dropna()
serie_fechamento_diff

In [ ]:
serie_fechamento_diff.plot()

In [ ]:
validacao_distribuicao(serie_fechamento_diff)

In [ ]:
validacao_kpss(serie_fechamento_diff)

In [1231]:
def validacao_adfuller(serie):
  print(colored(':: Validação ADF (Dickey-Fuller) ::', 'light_blue', attrs=["bold"]))
  result_adf = adfuller(serie)
  statistic_value = result_adf[0]
  p_value = result_adf[1]
  critical_values =  result_adf[4]
  valor_referencia = 0.05

  print(f"ADF Statistic: {statistic_value}")
  print(f'Valor-p do Teste ADF: {p_value}')

  table = PrettyTable(['Criticidade','Valor'])
  for key, value in critical_values.items():
    table.add_row([key, value])

  print(table)

  if p_value > valor_referencia:
    print(colored('Não rejeitar a Hipótese Nula: a série não é estacionária\n','red'))
  else:
    print(colored('Rejeitar a Hipótese Nula: a série é estacionária\n','green'))

In [ ]:
validacao_adfuller(serie_fechamento_diff)

- Conseguimos através da diferenciação transformar a serie em estácionaria, agora podemos avaliar o comportamento da serie novamente

In [ ]:
stats.probplot(serie_fechamento_diff, dist='norm', plot=plt)
plt.plot()

In [ ]:
sns.histplot(serie_fechamento_diff, kde=True)

# Decomposição da serie Diferencial

In [ ]:
serie_diff_decomposta = seasonal_decompose(serie_fechamento_diff, period=30)
serie_diff_decomposta.plot()
plt.tight_layout()

In [ ]:
serie_diff_decomposta = seasonal_decompose(serie_fechamento_diff, period=365)
serie_diff_decomposta.plot()
plt.tight_layout()

##  Avalição residuos

In [ ]:
sns.histplot(serie_diff_decomposta.resid, kde=True)
plt.show()

- Conseguimos avaliar visualmente que o residuo aparenta ter uma distribuição bem proxima do normal com uma concentração de valores bem proximas a zero, porem a cauda indica possíveis outliers

In [ ]:
plotar_medias_moveis(serie_fechamento_diff, 'Valores da bolsa de valores com média movel com diferenciação')

In [ ]:
plotar_medias_moveis(serie_fechamento_diff.loc[pd.Timestamp('2020-01-02'):pd.Timestamp('2023-12-29')], 'Valores da bolsa de valores com média movel com diferenciação de 2020 a 2023')

# Autocorrelação com diferenciação


## A Importância da Autocorrelação após a Diferenciação em Séries Temporais

**Entendendo a Diferenciação e a Autocorrelação**

Antes de discutir a importância, vamos revisar brevemente esses conceitos:

* **Diferenciação:** É uma técnica utilizada para tornar uma série temporal estacionária, removendo tendências ou sazonalidades. Ao calcular a diferença entre um valor e o anterior, eliminamos o componente de nível da série, tornando-a mais estável.
* **Autocorrelação:** Mede a relação linear entre os valores de uma série em diferentes pontos no tempo. Em outras palavras, indica se os valores passados da série influenciam os valores futuros.

**Por que a Autocorrelação após a Diferenciação é Importante?**

1. **Identificação de Modelos:**
   * **ARIMA:** A análise da autocorrelação (ACF) e da autocorrelação parcial (PACF) após a diferenciação é fundamental para identificar a ordem dos modelos ARIMA (AutoRegressive Integrated Moving Average). Esses modelos são amplamente utilizados para modelar e prever séries temporais.
   * **Outros Modelos:** A autocorrelação também auxilia na escolha de outros modelos, como modelos de suavização exponencial ou modelos de decomposição de séries temporais.

2. **Validação do Modelo:**
   * **Resíduos:** Após ajustar um modelo, os resíduos (a diferença entre os valores observados e os valores previstos pelo modelo) devem ser não autocorrelacionados. A autocorrelação nos resíduos indica que o modelo não capturou toda a informação da série e pode ser melhorado.

3. **Interpretação da Série:**
   * **Padrões:** A autocorrelação pode revelar padrões na série, como ciclos ou tendências residuais que não foram completamente removidos pela diferenciação.
   * **Causalidade:** Em alguns casos, a autocorrelação pode sugerir relações de causa e efeito entre os valores da série.

**Em Resumo**

A análise da autocorrelação após a diferenciação é uma etapa crucial na modelagem de séries temporais. Ela permite:

* **Selecionar o modelo adequado:** Identificando a ordem dos modelos ARIMA ou outros modelos apropriados.
* **Avaliar a qualidade do modelo:** Verificando se os resíduos são não autocorrelacionados.
* **Compreender a natureza da série:** Revelando padrões e relações entre os valores.

**Exemplo Prático:**

Imagine uma série temporal de vendas de um produto. Após aplicar a diferenciação, você observa uma autocorrelação significativa nos primeiros lags da ACF. Isso sugere que os valores de vendas atuais estão fortemente relacionados aos valores dos últimos períodos. Com base nessa informação, você pode ajustar um modelo ARIMA para prever as vendas futuras.

**Considerações Adicionais:**

* **Estacionaridade:** A diferenciação é uma ferramenta importante para tornar a série estacionária, mas nem sempre é suficiente. Outras técnicas, como a transformação logarítmica, podem ser necessárias.
* **Outros Testes:** Além da autocorrelação, outros testes, como o teste de Dickey-Fuller aumentado (ADF), podem ser utilizados para verificar a estacionariedade da série.
* **Software:** Softwares estatísticos como R e Python oferecem funções para calcular a autocorrelação, ajustar modelos ARIMA e realizar outras análises de séries temporais.

**Em Conclusão**

A autocorrelação após a diferenciação é uma ferramenta poderosa para analisar e modelar séries temporais. Ao entender a importância dessa análise, você estará mais bem preparado para tomar decisões mais precisas e informadas com base nos dados.

**Gostaria de aprofundar algum desses tópicos ou tem alguma outra pergunta?**

**Possíveis tópicos para aprofundar:**

* Cálculo da autocorrelação e autocorrelação parcial
* Interpretação dos gráficos de ACF e PACF
* Escolha da ordem dos modelos ARIMA
* Outros modelos para séries temporais
* Aplicações práticas da análise de séries temporais

**Observação:** Se você tiver dados específicos, posso te auxiliar na análise. 


In [ ]:
plot_acf(serie_fechamento_diff)
plot_pacf(serie_fechamento_diff)
plt.show()

- A autocorrelação da diferenciação apresentou um resultado satisfatórios, pois foi possivel notar que a partir do indicador 2 a correlação se manteve proximo ao intervalo de confiança, exceto por um item fora, porem aceitavel, assim como a correlação parcial tambem teve seus bon resultados considerendo-se que tambem a partir do 2 ponto se manteve dentro ou bem proximo do intervalo de confiança. Com esse resultado já é possível utilizar um modelo ARIMA com ordem 2 baseado nesse resultado

# ARIMA NOVO

In [ ]:
modelo_novo = auto_arima(serie_fechamento_diff, d=1, seasonal=True, m=12, stepwise=True)
print(modelo_novo.summary())

seasonal_order = modelo_novo.seasonal_order
print(f'Detected Seasonal Order: {seasonal_order}')

# Aplicação do Modelo ARIMA

### AutoRegressive Integrated Moving Average (ARIMA)
*É um modelo estatístico amplamente utilizado para analisar e prever séries temporais. As letras que compõem essa sigla possuem significados específicos:*
* AR (AutoRegressiva): Indica que o valor da série em um determinado momento é uma função linear de seus valores anteriores. Ou seja, os valores passados influenciam os valores futuros.
* I (Integrada): Refere-se ao processo de diferenciação aplicado à série temporal. A diferenciação é uma técnica utilizada para tornar a série estacionária, removendo tendências ou sazonalidades.
* MA (Média Móvel): Indica que o valor da série em um determinado momento é uma função linear dos erros aleatórios (ruídos) ocorridos em momentos anteriores



*Um modelo ARIMA é representado por três números: ARIMA(p,d,q).*
* p: Ordem do processo autoregressivo. Indica o número de períodos anteriores que são usados para prever o valor atual.
* d: Grau de diferenciação. Indica o número de vezes que a série é diferenciada para torná-la estacionária.
* q: Ordem do processo de médias móveis. Indica o número de termos de erro anteriores que são incluídos no modelo.

- Considerando os valores encontrados nos resultados anterires, iremos começar testando o modelo com os seguintes valores p = 2, d = 1, e q = 1

- sendo *p = 2*  que o valor atual vai depender dos 2 valores anteriores, *d = 1* porque queremos que ele efetue a diferenciação uma unica vez e  *q = 1*  que o erro atual depende do erro anterior

In [ ]:
#melhor 2,1,2
p,d,q =  2,1,2


modelo_arima = ARIMA(serie_fechamento_diff, order = (p,d,q))

In [ ]:
resultado = modelo_arima.fit()
print(resultado.summary())

In [ ]:
resid = resultado.resid
model_garch = arch.arch_model(resid, mean='Zero', vol='GARCH', p=p, q=q).fit()

In [ ]:
data_inicio, data_fim = '2024-03-12','2024-04-10'
forecast_arima = resultado.forecast(steps=30)
forecast_garch = model_garch.forecast(horizon=30)
forecast = forecast_arima + np.sqrt(forecast_garch.variance.values[-1, :]) * np.random.normal(size=30)
serie_prevista_garch = pd.Series(data=forecast.values, index=pd.date_range(data_inicio, data_fim))


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(serie_fechamento_diff, label='REAL')
plt.plot(serie_prevista_garch, label='Previsão ARIMA + GARCH')
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot(serie_fechamento_diff.loc[pd.Timestamp('2023-01-02'):pd.Timestamp('2024-03-12')], label='REAL')
plt.plot(serie_prevista_garch, label='Previsão ARIMA + GARCH')
plt.legend()
plt.show()

In [ ]:
serie_fechamento_diff[-12:]

In [ ]:
antigo = resultado.get_prediction(start=-12)
predito = antigo.tvalues

rmse_arima = root_mean_squared_error(serie_fechamento_diff[-12:].values, predito)
rmse_arima

In [ ]:
predito

In [ ]:
serie_fechamento_diff

In [ ]:
df_new = pd.DataFrame(serie_fechamento_diff[-12:],columns=['real'])
df_new['predito'] = predito

df_new

plt.plot(df_new['real'])
plt.plot(df_new['predito'])

In [ ]:
config = antigo.conf_int()
config

In [ ]:
import yfinance as yf

df_real_ibovesp = yf.download(tickers='^BVSP',interval='1d',start='2024-03-12',end='2024-04-10')

df_real_ibovesp.head


In [ ]:
df_real_ibovesp.info()

In [ ]:
serie_real_ibovesp = pd.Series(data=df_real_ibovesp['Close'], index=df_real_ibovesp.index)
serie_real_ibovesp  = serie_real_ibovesp / 1000 #Ajustando casas de
serie_real_ibovesp.head()


In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import acf, q_stat

# ... (código para ajustar o modelo)
y_true = serie_real_ibovesp.values

# Calcular métricas
rmse = np.sqrt(root_mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

# Teste de Ljung-Box
acf_resid = acf(model_garch.resid)
q, pval = q_stat(acf_resid[1:], len(acf_resid[1:]))


print(rmse)
print(mae)

#
print(q,pval)

# ... (outros testes e análises)

In [ ]:
residuos_arima = resultado.resid
residuos_arima

In [ ]:
residuos_arima

In [ ]:
residuos_arima.plot()
plt.show()

In [ ]:
residuos_arima

In [ ]:
sns.histplot(residuos_arima, kde=True)
plt.show()

In [ ]:
validacao_distribuicao(residuos_arima)
validacao_kpss(residuos_arima)
validacao_adfuller(residuos_arima)

In [ ]:
plot_acf(residuos_arima)
plot_pacf(residuos_arima)
plt.show()

In [ ]:
residuos_arima

In [ ]:
plt.plot(serie_fechamento_diff, label='Série Real')
plt.plot(serie_fechamento_diff-residuos_arima,color='red', label='Resíduos', alpha=0.4)
plt.legend(loc='best')
plt.show()

In [ ]:
serie_2020_a_2023 = serie_fechamento_diff.loc[pd.Timestamp('2020-01-02'):pd.Timestamp('2023-12-29')]
residuos_arima_2020_a_2023 = residuos_arima.loc[pd.Timestamp('2020-01-02'):pd.Timestamp('2023-12-29')]
plt.title('Real x Residuo do modelo arima de 2020 a 2023')
plt.plot(serie_2020_a_2023, label='Série Real')
plt.plot(serie_2020_a_2023-residuos_arima_2020_a_2023,color='red', label='Resíduos', alpha=0.8)
plt.xlabel('Data')
plt.ylabel('Valor')
plt.legend(loc='best')
plt.show()

In [ ]:
resultado.fittedvalues

# Previsão de valores

In [ ]:
data_inicio = '2024-03-11'
data_fim = '2024-04-12'
previsao_arima = resultado.forecast(steps=len(pd.date_range(data_inicio, data_fim)))
previsao_arima

In [ ]:
serie_prevista = pd.Series(data=previsao_arima.values, index=pd.date_range(data_inicio, data_fim))
serie_prevista

In [ ]:
serie_prevista = pd.Series(data=previsao_arima.values, index=pd.date_range(data_inicio, data_fim))

ultimo_ano = resultado.fittedvalues.loc[pd.Timestamp('2023-01-02'):pd.Timestamp('2024-03-11')]

ultimo_ano.plot()

serie_prevista.plot()



# Auto Arima

In [ ]:
modelo_auto_arima = auto_arima(serie_fechamento_diff, 
                               d=1, 
                               start_p=0, 
                               start_q=0, 
                               max_p=3, 
                               max_q=3, 
                               seasonal=True, 
                               m=6,
                               D=1, 
                               start_P=1,
                               max_Q=2,
                               information_criterion='aic',
                               trace=True,
                               stepwise=True                               
                               )

In [ ]:
print(modelo_auto_arima.aic())

In [ ]:
resultado_auto_arima = modelo_auto_arima.fit(serie_fechamento_diff)
print(resultado_auto_arima.summary())

In [ ]:
residuo_auto_arima  = resultado_auto_arima.resid
residuo_auto_arima()

In [ ]:
residuo_auto_arima().plot()
plt.show()

In [ ]:
stats.probplot(residuo_auto_arima(), dist='norm', plot=plt)
plt.show()

In [ ]:
sns.histplot(residuo_auto_arima(), kde=True)

In [ ]:
validacao_distribuicao(residuo_auto_arima())
validacao_adfuller(residuo_auto_arima())
validacao_kpss(residuo_auto_arima())

In [ ]:
plot_acf(residuo_auto_arima())
plot_pacf(residuo_auto_arima())

plt.show()

In [ ]:
data_inicio = '2024-03-12'
data_fim = '2024-04-12'
previsao_auto_arima = resultado_auto_arima.predict(n_periods=len(pd.date_range(data_inicio, data_fim)))
previsao_auto_arima


In [ ]:
previsao_auto_arima.shape

In [ ]:
index=pd.date_range(data_inicio, data_fim)
index.shape


In [1284]:
previsao_auto_arima = previsao_auto_arima**3

In [1285]:
serie_prevista_auto_arima = pd.Series(data=previsao_auto_arima.values, index=pd.date_range(data_inicio, data_fim))

In [ ]:
plotarSerie(serie_fechamento_diff)
#plotarSerie(serie_fechamento-residuo_auto_arima(), legendas=['Serie Real','Residuo'])
plotarSerie(serie_prevista_auto_arima, legendas=['Serie Real','Residuo'])
plt.show()

In [ ]:
previsao_auto_arima = resultado_auto_arima.predict(n_periods=30)
previsao_auto_arima

In [ ]:
previsao_auto_arima.plot()

In [ ]:
modelo_auto_arima = auto_arima(serie_fechamento_diff, 
                               d=1, 
                               start_p=0, 
                               start_q=0, 
                               max_p=3, 
                               max_q=3, 
                               seasonal=True, 
                               m=6,
                               D=1, 
                               start_P=1,
                               max_Q=2,
                               information_criterion='aic',
                               trace=True               
                               )

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model_sar = SARIMAX(serie_fechamento_diff, order=(3,1,0), seasonal_order=(2,1,0,6))
resultado_sarimax = model_sar.fit()


In [ ]:
resultado_sarimax.summary()

In [ ]:
predict = resultado_sarimax.get_prediction(start=-12)
predict_media = predict.predicted_mean

predict_media.plot()

In [ ]:
predict.conf_int()

# XBoost

In [ ]:
serie_fechamento

In [ ]:
df_clone

In [1296]:
df_clone = df

In [1297]:
df_clone =  df_clone.set_index('Data')

In [1298]:
data_final_treino = df_clone.reset_index().iloc[int(len(df) * (1 - 0.98))].values


data_inicial_test = df_clone.reset_index().iloc[int(len(df) * (1 - 0.98))-1].values


In [ ]:
data_final_treino = data_final_treino[1]
data_final_treino

In [ ]:
data_inicial_test = data_inicial_test[1]
data_inicial_test

In [ ]:
df[df.index > 99]

In [ ]:
df[df.index < 100]

In [1303]:
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')

df = df.sort_values(by='Data', ascending=True)

In [1304]:
dias_teste = 30

In [ ]:
train, test = df.iloc[0:len(df)-dias_teste], df.iloc[len(df)-dias_teste+1:len(df)]
data_test_inicio = test["Data"].min()
data_test_inicio = str(data_test_inicio).split(" ")[0]
data_test_fim = test["Data"].max()
data_test_fim = str(data_test_fim).split(" ")[0]
print(f'Treino de {train["Data"].min()} até {train["Data"].max()}')
print(f'Test de {data_test_inicio} até {data_test_fim}')
print(f'Tamanho treino {train.shape}')
print(f'Tamanho treino {test.shape}')

In [1306]:
def create_feature(data_frame):
    data_frame['valor_fechamento']  = data_frame['Último']
    data_frame['abertura']  = data_frame['Abertura']
    data_frame['date'] = data_frame["Data"]
    data_frame['year'] = data_frame["Data"].dt.year
    data_frame['month'] = data_frame["Data"].dt.month
    data_frame['day'] = data_frame["Data"].dt.day
    data_frame['dayofweek'] = data_frame["Data"].dt.dayofweek
    return data_frame

In [ ]:
train = create_feature(train)
test = create_feature(test)

In [ ]:
train.head()

In [ ]:
test.head()

In [1310]:
feature_names = ["year", "month", "day", "dayofweek","abertura"]
target = "valor_fechamento"

In [1311]:
X_train, y_train = train[feature_names], train[target]
X_test, y_test = test[feature_names], test[target]

In [1312]:
from  sklearn.metrics import recall_score

In [ ]:
X_test

In [ ]:
df_real_ibovesp = yf.download(tickers='^BVSP',interval='1d',start=data_test_inicio,end=data_test_fim) / 1000
df_real_ibovesp

In [1315]:
reg = xgb.XGBRegressor(objective="reg:squarederror")
reg.fit(X_train, y_train)

preds = reg.predict(X_test)

In [1316]:
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = root_mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return mae, mse, mape


def print_metrics(metrics):
    mae, mse, mape = metrics
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"MAPE: {mape:.2f} %")

In [ ]:
metrics = calculate_metrics(y_true=preds, y_pred=y_test.values)
print_metrics(metrics)

In [ ]:
mape = metrics[2]

print(f'Acuracia {100 - mape:.2f} %')

In [ ]:
X_test["data"] = X_test["year"].map(str) + "-" + X_test["month"].map(str) + "-" + X_test["day"].map(str)
X_test["data"] = pd.to_datetime(X_test['data'])
X_test

In [ ]:

pd.concat([df_real_ibovesp, pd.Series(data=preds,index=X_test['data'])], axis=1)

In [ ]:
pd.Series(data=preds,index=X_test['data']).plot()
df_real_ibovesp['Close'].plot()
plt.show()
#recall = recall_score(y_test, preds)

#print(f"Recall Score: {recall:.2f}")

# Prophet

In [1322]:
train_prophet = train.rename(columns={"date":"ds", "valor_fechamento":"y"})
test_prophet = test.rename(columns={"date":"ds", "valor_fechamento":"y"})

In [ ]:
model_profet = Prophet(daily_seasonality=True)
model_profet.add_regressor("abertura")
model_profet.fit(train_prophet)

In [ ]:
future = model_profet.make_future_dataframe(periods=len(test))
future

In [ ]:
future['abertura'] = pd.concat([train['abertura'], test["abertura"]], ignore_index=True)
future

In [ ]:
forecast_prophet = model_profet.predict(future)
forecast_prophet.head()

In [ ]:
preds_prophet = forecast_prophet[['ds','yhat']].tail(len(test))
preds_prophet

In [ ]:
preds_prophet = preds_prophet.set_index("ds")
preds_prophet

In [ ]:
y_test_prophet = test_prophet.set_index('ds')['y']
y_test_prophet.head()

In [ ]:
metrics_pr = calculate_metrics(y_test_prophet, preds_prophet['yhat'])
print('Prophet Metrics') 
print_metrics(metrics_pr)

# Sarimax